In [2]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# ==== 1. 載入原始Keras模型 (.h5) ====
model = tf.keras.models.load_model('MobileNetV2.h5')

# ==== 2. 設定校準資料集（Calibration Dataset）====

# 注意：這裡必須提供一小組訓練資料作為量化的參考 (約100~500張即可)
def representative_dataset_gen():
    dataset_dir = 'Garbage'  # 請替換為你的資料夾路徑
    datagen = ImageDataGenerator(rescale=1./255)
    calib_generator = datagen.flow_from_directory(
        dataset_dir,
        target_size=(224, 224),
        batch_size=16,
        class_mode=None,
        shuffle=True
    )

    for i in range(500):
        img = next(calib_generator)
        yield [img]

# ==== 3. 建立 TFLiteConverter 轉換器 ====
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# 設定為全整數量化 (int8)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# 設定輸入/輸出張量皆為int8
converter.target_spec.supported_types = [tf.int8]  # float16 或 int8
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

# ==== 4. 轉換模型並儲存 ====
tflite_quant_model = converter.convert()

# 儲存為.tflite檔案
with open('int8_MobileNetV2.tflite', 'wb') as f:
    f.write(tflite_quant_model)

print("✅ 轉換完成，已儲存為 model_int8.tflite")


INFO:tensorflow:Assets written to: C:\Users\willy\AppData\Local\Temp\tmpz32812ty\assets


c:\Users\willy\anaconda3\envs\tensorflow_gpu\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Found 8079 images belonging to 4 classes.
✅ 轉換完成，已儲存為 model_int8.tflite


In [3]:
import tensorflow as tf
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, f1_score, confusion_matrix, ConfusionMatrixDisplay

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [5]:
# === 1. 載入 int8 TFLite 模型 ===
interpreter = tf.lite.Interpreter(model_path="int8_MobileNetV2.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape'][1:3]


# 取得量化參數 (scale, zero_point)
input_scale, input_zero_point = input_details[0]['quantization']

In [6]:
# === 2. 定義影像預處理函式 ===
def preprocess_img(img_path, input_shape, scale, zero_point):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, input_shape)
    img = img.astype(np.float32) / 255.0  # 先歸一化到 [0,1]

    # 將資料量化為 int8
    img = img / scale + zero_point
    img = np.clip(img, -128, 127).astype(np.int8)
    img = np.expand_dims(img, axis=0)
    return img


In [7]:
# === 3. 載入測試資料 ===
def load_test_data(img_dir):
    X = []
    y = []
    class_names = sorted(os.listdir(img_dir))  # 每個子資料夾為一個類別
    class_to_idx = {name: idx for idx, name in enumerate(class_names)}

    for class_name in class_names:
        class_folder = os.path.join(img_dir, class_name)
        if not os.path.isdir(class_folder):
            continue
        for fname in os.listdir(class_folder):
            if fname.lower().endswith(('.jpg', '.png', '.jpeg')):
                img_path = os.path.join(class_folder, fname)
                img = preprocess_img(img_path, input_shape, input_scale, input_zero_point)
                if img is not None:
                    X.append(img)
                    y.append(class_to_idx[class_name])
    return X, y, class_names
test_folder = "trashbox"  # <-- 請修改為你的測試資料夾路徑
X_test, y_true, class_names = load_test_data(test_folder)

In [8]:
# === 4. 推論模型 ===
y_pred = []
for img in X_test:
    interpreter.set_tensor(input_details[0]['index'], img)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])
    pred = np.argmax(output, axis=-1)[0]
    y_pred.append(pred)

In [9]:
# === 6. 輸出分類報告 (包含Accuracy, Precision, Recall, F1-score) ===
accuracy = accuracy_score(y_true, y_pred)
print(f"準確率: {accuracy}")
print(classification_report(y_true, y_pred, target_names=class_names, digits=4))


準確率: 0.7095839370730698
              precision    recall  f1-score   support

          BG     0.7771    1.0000    0.8746       373
       Metal     0.4246    0.7586    0.5445      1562
          PG     0.8888    0.5492    0.6789      3696
       Paper     0.7957    0.8107    0.8031      4031

    accuracy                         0.7096      9662
   macro avg     0.7215    0.7797    0.7253      9662
weighted avg     0.7706    0.7096    0.7166      9662

